# Baseline AML Risk Modeling

## Overview
This notebook consolidates all engineered features into a **baseline AML risk model**, focusing on ranking performance under extreme class imbalance.

---

## Inputs
- Consolidated customer feature table
- `labels.csv` (partial supervision)

## Outputs
- Train / validation splits
- Logistic Regression, KNN, tree-based baselines
- ROC-AUC, PR-AUC metrics
- Customer-level risk scores

## Goal
- Establish a reproducible AML baseline
- Validate usefulness of rule-derived features
- Produce analyst-ready risk rankings

## Notes
- Precision is expected to be low
- Ranking and explainability are primary objectives
